In [1]:
from datetime import datetime
from collections import Counter

# Data management
import pandas as pd

# Data preprocessing and trasformation (ETL)
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, MaxAbsScaler, FunctionTransformer, Binarizer, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml, load_iris, make_moons, make_classification


# Math and Stat modules
import numpy as np
from scipy.stats import sem
from random import choice

# Supervised Learning
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict, KFold, StratifiedKFold, RepeatedKFold, ShuffleSplit, StratifiedShuffleSplit, learning_curve, validation_curve
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.base import BaseEstimator
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, precision_recall_curve, roc_curve, accuracy_score
from sklearn.dummy import DummyClassifier
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.ensemble import RandomForestClassifier

# Unsupervised Learning

# Visualization
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

# Network
import networkx as nx

In [3]:
# Il Problema di Link Prediction può essere modellato come un Problema di Classificazione

# Per farlo tramite Supervised Learning, occorre avere a disposizione una Rete Annotata Temporalmente (in cui a ogni
# arco sia associato un istante temporale)

FB_Temp = nx.Graph()

with open('../data/facebook/fb-wosn-friends.edges') as f:
    f.readline()
    f.readline()
    for line in f:
        u, v, _, ts = [int(e) for e in line.strip().split()]
        timestamp = datetime.fromtimestamp(ts if ts != 0 else 1129788114) # 20 ottobre 2005
        FB_Temp.add_edge(u, v, time=timestamp)

In [4]:
FB_Temp.order(), FB_Temp.size()

(63731, 817090)

In [6]:
# Definiamo due periodi temporali:
# t0 = 01/01/2004, t0' = 01/06/2008 <-- Grafo di Training
# t1 = 01/06/2008, t1' = 01/10/2008 <-- Grafo di Test

data_soglia, data_soglia_finale = datetime(2008,6,1), datetime(2008,10,1)

In [7]:
FB_train, FB_test = nx.Graph(), nx.Graph()
for u,v, info in FB_Temp.edges(data=True):
    if info['time'] < data_soglia:
        FB_train.add_edge(u,v)
    elif info['time'] < data_soglia_finale:
        FB_test.add_edge(u,v)

In [8]:
FB_train.order(), FB_train.size()

(48983, 475483)

In [9]:
FB_test.order(), FB_test.size()

(38481, 158361)

In [10]:
# Costruiamo poi i Datapoint:
# - con Label = 1 (gli Archi di FB_test)
# - con Label = 0 (scelti a caso tra quelli non in FB_test (e nemmeno in FB_train ovviamente))

test_nodes = list(FB_test.nodes())
test_edges = np.array([{u, v} for u, v in FB_test.edges()])

positive_datapoints = list()
for u, v in np.random.choice(test_edges, replace=False, size=12000): # Pick 12k random edges
    if FB_train.has_node(u) and FB_train.has_node(v):
        positive_datapoints.append((u, v))


negative_datapoints = list()
while len(negative_datapoints) < len(positive_datapoints):
    u, v = np.random.choice(test_nodes, replace=False, size=2)
    if FB_train.has_node(u) and FB_train.has_node(v) and not FB_train.has_edge(u, v) and not FB_test.has_edge(u, v):
        negative_datapoints.append((u, v))

len(positive_datapoints), len(negative_datapoints)

(10749, 10749)

In [18]:
# Dobbiamo ora costruire il DataFrame per l'Apprendimento.
# La tabella verrà riempita con una serie di Feature rilevanti estraibili dagli Archi

df = pd.DataFrame()
all_datapoints = positive_datapoints + negative_datapoints

# Feature 1: Jaccard Coefficient (for Neighbourhood Overlap)
df['jaccard'] = [p for _, _, p in nx.jaccard_coefficient(FB_train, all_datapoints)]
# Feature 2: Resource Allocation Index
df['rai'] = [p for _, _, p in nx.resource_allocation_index(FB_train, all_datapoints)]
# Feature 3: Adamic-Adar Index
df['aai'] = [p for _,_,p in nx.adamic_adar_index(FB_train, all_datapoints)]
# Feature 4: Preferential Attachment Index
df['pref'] = [p for _,_,p in nx.preferential_attachment(FB_train, all_datapoints)]

# Label
df['label'] = np.concatenate((np.ones(len(positive_datapoints)), np.zeros(len(negative_datapoints))))

df

,jaccard,rai,aai,pref,label
0,0.183099,0.224882,3.142035,1568,1.0
1,0.008696,0.166667,0.558111,115,1.0
2,0.055556,0.142857,0.513898,70,1.0
3,0.000000,0.000000,0.000000,8,1.0
4,0.029630,0.090845,2.423002,41666,1.0
...,...,...,...,...,...
21493,0.000000,0.000000,0.000000,272,0.0
21494,0.000000,0.000000,0.000000,138,0.0
21495,0.000000,0.000000,0.000000,450,0.0
21496,0.016949,0.040894,0.869279,14399,0.0


In [22]:
# Da questo momento in poi è un generico problema di Supervised Learning

X = df[['jaccard', 'rai', 'aai', 'pref']]
y = df['label']

# Preprocessing
ct = ColumnTransformer([
    ('minmax', MinMaxScaler(), ['jaccard', 'rai', 'aai', 'pref'])
])

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

# Pipeline
predictor = Pipeline([
    ('preprocessing', ct),
    ('classifier', LogisticRegression(penalty=None))
])

predictor.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('minmax', MinMaxScaler(),
                                                  ['jaccard', 'rai', 'aai',
                                                   'pref'])])),
                ('classifier', LogisticRegression(penalty=None))])

In [25]:
y_prediction = predictor.predict(X_test)

In [26]:
accuracy_score(y_test, y_prediction)

0.8272868217054263

In [27]:
f1_score(y_test, y_prediction)

0.7986261749819233

In [30]:
confusion_matrix(y_prediction, y_test)

array([[3127, 1032],
       [  82, 2209]])

In [31]:
# Con Logistic Regression possiamo vedere come il modello ha utilizzato le Feature, in particolare quali Feature siano stati più decisive

predictor['classifier'].coef_

array([[50.0294412 , 47.25960252, 52.39890364, -1.86403766]])